<h1>SASSIE Shipboard ADCP Data Use and Visualization</h1>
<p> Shipboard ADCP data was collected throughout the SASSIE campaign from the R/V Woldstad. Due to a heading error on the ADCP, only vertical sheer data are avalible. ADCP data are downloaded and plotted using the code below! <b>Please run the 'Data Download and Metadata Viewing' and 'Supporting Code' sections in order before running the 'Figure Making Code'.</b> </p>
    
<p>The aim of this notebook is to assist the end user in exploratory data analysis by downloading the SASSIE data from NASA's PODAAC, opening the dataset and displaying it's associated metadata, and creating a few visualizations. This notebook was created by Elizabeth Westbrook. For questions and trouble shooting, please email westbrooke@uncw.edu.</p>

# Credential Entry
In this section, you shoud enter your EarthData username and password. <b> DO NOT enter usernames and passwords here which are sensitive.</b> If you do not already have an EarthData account, you can create one <a href="https://urs.earthdata.nasa.gov/">here</a> . </p>
    <p>Please enter your EarthData credentials below: </p>

In [ ]:
#Enter Earthdata Credentials: 
username = 'your_username'
password = 'your_password'

# Data Download and Metadata Viewing

The code in this section will download the dataset from PO.DAAC and open it as an xarray object for metadata and variable attribute viewing.

In [ ]:
import numpy as np
import xarray as xr
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd
import requests
import cmasher as cmr

<h2>Download The Shipboard ADCP data from PO.DAAC</h2>
<p> All data from the SASSIE campaign is stored on NASA's PO.DAAC. The code in this section of the notebook will download SASSIE Shipboard ADCP data from PO.DAAC, which is accessed through EarthData. 

In [ ]:
##LOCAL DIRECTORY TO SAVE SHIPBOARD ADCP DATA
dir_in = 'Data/ADCP/'

The next block of code creates the directory specified above and downloads the shipboard ADCP file to your binder session if it has not already been downloaded.<b> To download the dataset to your local disk from here, right click on the file you want to download and click 'download'. </b>

In [ ]:
#if the file has not already been downloaded, it is downloaded here: 
if not os.path.isfile(dir_in+'SASSIE_Fall_2022_ADCP_Ocean_current_sheer.nc'):
    os.makedirs(dir_in)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_ADCP_V1/SASSIE_Fall_2022_ADCP_Ocean_current_sheer.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(dir_in+'SASSIE_Fall_2022_ADCP_Ocean_current_sheer.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + dir_in+'SASSIE_Fall_2022_ADCP_Ocean_current_sheer.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard ADCP file is already in binder directory')

<h2> View The Metadata Inside the Shipboard ADCP File</h2>

The netCDF file has global metadata attributes and attributes associated with each variable. This next block will load data and metadata of the netCDF file into an xarray object (ds). <br> The data set will then be displayed in a clickable HTML format. 

In [ ]:
#See information about the entire dataset
file = dir_in + 'SASSIE_Fall_2022_ADCP_Ocean_current_sheer.nc'
ds = xr.open_dataset(file)
ds

# Supporting Code 
The code in this section provides a set up for the figure making code below by defining directories for data and figures and creating functions that will be called to actually map the data

<h3>Create a Directory to Save Figures</h3>

In [ ]:
#LOCAL DIRECTORY TO SAVE FIGURES
fig_dir ='Figures/ADCP/'
#FIGURE DIR 
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

<h3>Define a Colormap and Label for Each Variable in the File</h3>

Within SASSIE's collection of jupyter notebooks, the colormaps used for each variable are held as consistant as possible across all datasets. This function defines the colormap and a label for the variable of interest. 

In [ ]:
#DEFINES COLORMAPS AND LABELS OF EACH VARIABLE IN THIS DATA SET
def define_variable_attributes(var):
    if var =='shear':
        colormap = 'PiYG'
        var_label = 'Vertical Shear (1/s)'
    if var == 'latitude':
        colormap = 'jet'
        var_label = 'Latitude ($^{\circ}$N)'
    if var == 'longitude':
        colormap = 'jet'
        var_label = 'Longitude ($^{\circ}$E)'
    if var == 'time':
        colormap = 'jet'
        var_label = 'Date'
    return colormap,var_label

<h3>Are you exploring this entire data set, or just the data collected durring a specific time period?</h3>

<p> In order to efficiently accomplish the data collection goals of the SASSIE campaign, the cruise was broken up into five destinct plays. 
    <br>
    <br>Play 1 - Ice and Open Water Survey
    <br>Play 2 - Zig-Zag Open Water Survey 
    <br>Play 3 - Ice Survey
    <br>Play 4 - Drifter-Following Survey Boxes
    <br>Play 5 - Repeated Ice to Open Water Transect</p>
    <p>The following block of code identifies the start and end datetimes of each of these plays and puts the start dates and end dates into lists for easier acess later. </p>

In [ ]:
##Identify the start and end times of each play
play1_start = datetime(2022,9,8,8) 
play1_end = datetime(2022,9,13)
play2_start = datetime(2022,9,14)
play2_end = datetime(2022,9,16)
play3_start = datetime(2022,9,16)
play3_end = datetime(2022,9,18)
play4_start = datetime(2022,9,19)
play4_end  = datetime(2022,9,23)
play5_start = datetime(2022,9,25)
play5_end = datetime(2022,9,29)

#subset the dataset into each play. 
play1_data =ds.sel(time=slice(play1_start,play1_end))
play2_data =ds.sel(time=slice(play2_start,play2_end))
play3_data =ds.sel(time=slice(play3_start,play3_end))
play4_data =ds.sel(time=slice(play4_start,play4_end))
play5_data =ds.sel(time=slice(play5_start,play5_end))
campaign_data = ds.sel(time=slice(play1_start,play5_end))

#Create lists of start dates and end dates
play_start_dates = [play1_start,play2_start,play3_start,play4_start,play5_start,play1_start]
play_end_dates = [play1_end,play2_end,play3_end,play4_end,play5_end,play5_end]
play_data_all = [play1_data,play2_data,play3_data,play4_data,play5_data,campaign_data]


<h3>Define a Function to Create a Map of the Study Area</h3>

The following function creates a map of the SASSIE study area, which is defined by minimum and maximum lat/lon values. These ranges can be changed later when the function is called to zoom in/out on the study area.

In [ ]:
def map_study_area(latmin = 70, latmax =74,lonmin=-157,lonmax=-140):
    
    global fig 
    global ax
    
    #create the map as a figure, set the lat and lon ranges, and add land + river data:
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50',zorder=1)
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')
    #Add lat and lon gridlines and labels:
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3) #draw_labels=True gives lat labels.
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,75,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-170,-140,2))
    gl.top_labels = False
    gl.bottom_labels = True 
    gl.right_labels = False
    
    #Add markers for reference cities on the coast:
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)
        

<h3>Define a Function to Create a 3D Grid of Latitude, Longitude, and Depth</h3>

The following function creates a 3D grid for plotting 3 dimensional cast data from the castaway CTD. 

In [ ]:
def make_3d_grid(latmin,latmax,lonmin,lonmax,dmin,dmax):    
        global fig 
        global ax
        
        fig = plt.figure(figsize=(10,8))
        ax = plt.axes(projection='3d',computed_zorder=False)
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False    


        ax.set_xlim(lonmin, lonmax); ax.set_ylim(latmin,latmax); ax.set_zlim(dmin,dmax);
        ax.get_xaxis().get_major_formatter().set_useOffset(False)
        xticks = list(np.linspace(lonmin,lonmax,4))
        ax.set_xticks(np.around(xticks,2))
        ax.set_xticklabels(np.around(xticks,2),rotation=30, ha='center',va='center', minor=False,size='small')

        yticks = list(np.linspace(latmin,latmax,4))
        ax.set_yticks(np.around(yticks,2))
        ax.invert_yaxis()
        ax.set_yticklabels(np.around(yticks,2),rotation=-15,va='bottom', minor=False,size='small')

        zticks = list(np.linspace(dmin,dmax,4))
        ax.set_zticks(np.around(zticks,2))
        ax.set_zticklabels(np.around(zticks,2),ha='center',va='center', minor=False,size='small')

        ax.set_xlabel('Longitude',fontsize=13,labelpad=10,fontweight='bold')
        ax.set_ylabel('Latitude',fontsize=13,labelpad = 7,fontweight='bold')
        ax.set_zlabel('Depth (m)',fontsize=13,labelpad = 16,fontweight='bold')
        ax.view_init(195,280)
        fig.canvas.draw()
        plt.tight_layout()

<h3>Configure Supporting Data to Add to Maps</h3>

<p>The functions for viewing and plotting this data set below have options to include bathymetr and/or shiptrack data to add context to maps. If you are using these options, run
    <br>the following code blocks to:
    <br>1. Create a directory for SASSIE Ship Track data and acess bathymetry data from NOAA
    <br>2. Define functions that add these data to your map when called.</p>

<h4>Create Directory for and Download Shiptrack Data</h4>

In [ ]:
## DIRECTORY TO SHIP TRACK DATA
ship_dir =  'Data/TSG/' 

#DOWLOAD SHIPTRACK DATA
if not os.path.isfile(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc'):
    os.makedirs(ship_dir)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_TSG_V1/SASSIE_Fall_2022_Shipboard_TSG.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if (r.status_code == 200) & (r.ok):
                    with open(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard TSG file is already in local directory')

<h4> Access NOAA Bathymetry Data</h4>

In [ ]:
#READ IN TOPOGRAPHY/BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url) 

<h4>Define a Function to Index Relevant Bathymetry Data and Add it to the Map</h4>
This function will index bathymetry data from NOAA within the appropriate spatial range and add it to the map. 


In [ ]:
def add_bathy_data(latmin,latmax,lonmin,lonmax):
        topoin = etopodata.rose.values[0:-1:5,1:-1:5]
        lons = etopodata.etopo2_x.values[0:-1:5]
        lats = etopodata.etopo2_y.values[0:-1:5]
        lons_in_range = lons[np.where((lons >lonmin-1) & (lons<lonmax+1))]
        lats_in_range = lats[np.where((lats >latmin-1) & (lats<latmax+1))]
        topo_in_range = np.squeeze(topoin[np.squeeze(np.where((lats >latmin-1) & (lats<latmax+1))),:][:,np.where((lons >lonmin-1) & (lons<lonmax+1))])
        [bathy_lon,bathy_lat] = np.meshgrid(lons_in_range,lats_in_range)
        
        bathy = ax.contour(bathy_lon,bathy_lat,topo_in_range,np.arange(-6000,-1000,300),transform=cartopy.crs.PlateCarree(),cmap='gray',alpha = 0.2,zorder = 0)

<h4>Define a Function to Add Shiptrack Data to a Map</h4>
This function will pull the lat/lon data from the SASSIE Shipboard TSG file and put it onto a map o a 3D grid. 

In [ ]:
def add_ship_track(mode,start_date,end_date):
        ds_ship = xr.open_dataset(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc')
        if mode == 'grid':
            ds_ship = ds_ship.sel(time=slice(start_date,end_date))
        ship_time = np.squeeze(ds_ship['time'])
        ship_lat = np.squeeze(ds_ship['latitude'])
        ship_lon = np.squeeze(ds_ship['longitude'])
        if mode =='map':
            track = ax.plot(ship_lon, 
                         ship_lat,linewidth = 0.5,
                         c='black',
                           transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)
        if mode == 'grid':
            track = ax.plot(ship_lon, 
                         ship_lat,
                         c='black',label = 'Ship Track',zorder=2)

# Figure Making Code 

<h2> Mapping the Time and Location of ADCP Data Collection</h2>

The map_adcp_track function shows the locations of ADCP data collection throughout the campaign on a map, colored by time. Various features of this function: 
    <br>2. This function adds bathymetry contours showing the position of the continental shelf by default for reference. 

In [ ]:
#PLOT THE LOCATION OF THE ADCP DATA COLLECTION ON THE SHIP TRACK 
def map_adcp_track(ship_track=True, bathymetry_data=True):
 
    #LOAD CASTAWAY CTD DATA
    time = np.squeeze(ds['time'])
    lat = np.squeeze(ds['latitude'])
    lon = np.squeeze(ds['longitude'])
    
    #Define the boundaries of the map:
    latmin = 70
    latmax =74
    lonmin=-157
    lonmax=-144
    
    #create the map:
    map_study_area(latmin=latmin,latmax=latmax,lonmin=lonmin,lonmax=lonmax)
    
    #Add a title: 
    ax.set_title('Track of the R/V Woldstad',fontsize=22,pad=1) 
    
    #add optional map add-ons:
    if bathymetry_data == True: 
        add_bathy_data(latmin,latmax,lonmin,lonmax)   
    if ship_track == True:
        add_ship_track('map',play1_start,play5_end)
     
    #APPLY CTD TIME AND LOCATION DATA
    casts = ax.scatter(lon,lat,s = 0.5,
                       c = time,cmap = cmr.neon,
                       transform=cartopy.crs.PlateCarree(),zorder=2,
                      vmin=min(time.values), vmax=max(time.values))
    #Add a color bar:
    cbar = fig.colorbar(casts, ax=ax, orientation="horizontal", pad=0.1)
    cbar.set_label(label='Date',size='large',weight='bold')
    cbar_tick_array=(np.linspace(time[0].astype('int64'),time[-1].astype('int64'),5))
    cbar.set_ticks(cbar_tick_array)
    cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)
    
    #Add a legend:
    plt.legend(loc=2)

    ##SAVE FIGURE 
    if not os.path.exists(fig_dir+'map/'):
        os.makedirs(fig_dir+'map/')
    
    print('Saving Output Image:  '+fig_dir+'map/adcp_map.png')
    plt.savefig(fig_dir+'map/adcp_map.png',dpi='figure',format='png',bbox_inches='tight')
    

In [ ]:
map_adcp_track()

<h3>Three Dimensional Representation of Vertical Profile Data</h3>
Use the code in this section to plot the Underway CTD profile locations and depths on a 3D grid, colored by a physical measurement variable from the file (temperature, salinity, density, etc.)

In [ ]:
##USE TO PLOT THE PROFILES TAKEN WITHIN A DATE RANGE 
def ADCP_profiles(start_date,end_date,var,ship_track=True):
   # try:
    #LOAD Underway CTD DATA
    selected_data = ds.sel(time=slice(start_date,end_date))
    adcp_lat = selected_data['latitude']
    adcp_lon = selected_data['longitude']
    adcp_depth = selected_data['depth']
    adcp_var = selected_data[var]
    
    #put lat, lon, and depth in 2D format:
    [depth,lat] = np.meshgrid(adcp_depth,adcp_lat)
    [depth,lon] = np.meshgrid(adcp_depth,adcp_lon)

    #define colormap and label for variable of choice:
    colormap,var_label = define_variable_attributes(var)

    #define latitude and longitude ranges for the 3D grid. 
    latmin = min(adcp_lat.values)
    latmax = max(adcp_lat.values)
    lonmin = min(adcp_lon.values)
    lonmax = max(adcp_lon.values)
    pmin = min(adcp_depth.values)
    pmax = max(adcp_depth.values)+0.5

    #make the 3d grid for plotting: 
    make_3d_grid(latmin,latmax,lonmin,lonmax,0,pmax)

    #add a title to the grid:
    ax.set_title('ADCP'+' '+var_label+' '+'Readings from'+' '+str(start_date.year)+'/'+str(start_date.month).zfill(2)+'/'+str(start_date.day).zfill(2)+' '+str(start_date.hour).zfill(2)+':'+str(start_date.minute).zfill(2)+' to '+str(end_date.year)+'/'+str(end_date.month).zfill(2)+'/'+str(end_date.day).zfill(2)+' '+str(end_date.hour).zfill(2)+':'+str(end_date.minute).zfill(2),fontsize=15,pad=None)

    #Add the ship track to show the path of the casting:
    if ship_track==True:
        add_ship_track('grid',start_date,end_date)

    ##APPLY CTD DATA   
    a = ax.scatter(lon,lat,depth,c=(adcp_var.values),cmap = colormap,s=2,zorder=1,vmin = -0.05, vmax = 0.05) 
    #add a colorbar 
    fig.colorbar(a,label = var_label)

    ## SAVE THE FIGURE
    if not os.path.exists(fig_dir+var+'/'):
        os.makedirs(fig_dir+var+'/')

    print('Saving Output Image:  '+fig_dir+var+'/'+'ADCP'+var+str(start_date.year)+str(start_date.month).zfill(2)+str(start_date.day).zfill(2)+str(start_date.hour).zfill(2)+str(start_date.minute).zfill(2)+'_'+str(end_date.year)+str(end_date.month).zfill(2)+str(end_date.day).zfill(2)+str(end_date.hour).zfill(2)+str(end_date.minute).zfill(2)+'.png')
    plt.savefig(fig_dir+var+'/'+'ADCP'+var+str(start_date.year)+str(start_date.month).zfill(2)+str(start_date.day).zfill(2)+str(start_date.hour).zfill(2)+str(start_date.minute).zfill(2)+'_'+str(end_date.year)+str(end_date.month).zfill(2)+str(end_date.day).zfill(2)+str(end_date.hour).zfill(2)+str(end_date.minute).zfill(2)+'.png',dpi='figure',format='png',bbox_inches='tight')
    #except: 
     #   print('There is no ADCP data from ' + str(start_date.year)+'/'+str(start_date.month).zfill(2)+'/'+str(start_date.day).zfill(2)+' '+str(start_date.hour).zfill(2)+':'+str(start_date.minute).zfill(2)+' to '+str(end_date.year)+'/'+str(end_date.month).zfill(2)+'/'+str(end_date.day).zfill(2)+' '+str(end_date.hour).zfill(2)+':'+str(end_date.minute).zfill(2))
              
              
              

In [ ]:
ADCP_profiles(play4_start,play4_end,'shear')

<h2>Two Dimensional Representation of Vertical Profile Data</h2>
Use the code in this section to plot the Underway CTD profiles on a 2D time-depth axis, colored by a physical measurement variable from the file (temperature, salinity)

In [ ]:
##MAKES A 2D PLOT OF THE CTD VARIABLES ON TIME-DEPTH AXIS (NO INTERPOLATION OF LAT/LON)
def plot_ADCP_2D(start_time,end_time,var_min,var_max):
    try:
        
        var = 'shear'
        
        #Cut the CTD data set down to the specified time range:
        ds_limited = ds.sel(time=slice(start_time,end_time))

        #load the CTD data within this time range
        ADCP_time = ds_limited['time'].values
        ADCP_depth = ds_limited['depth'].values
        ADCP_var = ds_limited[var].values

        #put time and depth into a 2D format
        [depth,time] = np.meshgrid(ADCP_depth,ADCP_time)

        ##SET UP THE 2D PLOT 
        colormap,var_label = define_variable_attributes(var)

        dmin = min(ADCP_depth)
        dmax = max(ADCP_depth)
        tmin = min(ADCP_time)
        tmax = max(ADCP_time)

        fig = plt.figure(figsize=(10,4))

        plt.title('ADCP '+' '+var_label+' '+'Readings \nfrom '+str(start_time.year)+'/'+str(start_time.month).zfill(2)+'/'+str(start_time.day).zfill(2)+' to '+str(end_time.year)+'/'+str(end_time.month).zfill(2)+'/'+str(end_time.day).zfill(2),fontsize=15,pad=None)

        plt.xlim(tmin, tmax); plt.ylim(dmin,100);
        total_timedelta = tmax-tmin
        third_of_timedelta = total_timedelta/3
        plt.xticks([tmin,tmin+third_of_timedelta, tmin+ 2*third_of_timedelta, tmax])
        ytick = list(np.arange(0,100,15))
        plt.yticks(np.around(ytick,2))
        plt.xlabel('Date',fontsize=16)
        plt.ylabel('Depth(m)',fontsize=16)
        plt.gca().invert_yaxis()

        ##APPLY THE CTD DATA
        ax = plt.scatter(time,depth,s=1,c=ADCP_var,cmap=colormap,vmin = var_min,vmax=var_max)

        #add a colorbar
        fig.colorbar(ax).set_label(label = var_label,size=15,weight='bold')

        ##SAVE THE FIGURE:
        if not os.path.exists(fig_dir+var+'/2d/'):
            os.makedirs(fig_dir+var+'/2d/')

        print('Saving Output Image:  '+fig_dir+var+'/2d/'+'ADCP_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png')
        plt.savefig(fig_dir+var+'/2d/'+'ADCP_'+var+'_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png',dpi='figure',format='png',bbox_inches='tight')
    except: 
        print('There is no ADCP data from ' + str(start_date.year)+'/'+str(start_date.month).zfill(2)+'/'+str(start_date.day).zfill(2)+' '+str(start_date.hour).zfill(2)+':'+str(start_date.minute).zfill(2)+' to '+str(end_date.year)+'/'+str(end_date.month).zfill(2)+'/'+str(end_date.day).zfill(2)+' '+str(end_date.hour).zfill(2)+':'+str(end_date.minute).zfill(2))
       

In [ ]:
plot_ADCP_2D(play1_start,play5_end,-0.05,0.05)


## Two Dimensional Time Series plot, Colored by Depth

Use the code in this sectin to plot the ADCP data in more detail by ploting shear vs time, colored by depth. 

In [ ]:
##MAKES A 2D PLOT OF THE CTD VARIABLES ON TIME-DEPTH AXIS (NO INTERPOLATION OF LAT/LON)
def plot_ADCP_2D_depth(start_time,end_time):
    #try:
    #Cut the CTD data set down to the specified time range:
    ds_limited = ds.sel(time=slice(start_time,end_time))

    #load the CTD data within this time range
    ADCP_time = ds_limited['time'].values
    ADCP_depth = ds_limited['depth'].values
    ADCP_var = ds_limited['shear'].values

    #put time and depth into a 2D format
    [depth,time] = np.meshgrid(ADCP_depth,ADCP_time)

    ##SET UP THE 2D PLOT
    dmin = min(ADCP_depth)
    dmax = max(ADCP_depth)
    tmin = min(ADCP_time)
    tmax = max(ADCP_time)

    fig = plt.figure(figsize=(10,4))

    plt.title('ADCP Vertical Current Shear Readings \nfrom '+str(start_time.year)+'/'+str(start_time.month).zfill(2)+'/'+str(start_time.day).zfill(2)+' to '+str(end_time.year)+'/'+str(end_time.month).zfill(2)+'/'+str(end_time.day).zfill(2)+'\n Colored By Depth',fontsize=15,pad=None)

    plt.xlim(tmin, tmax); plt.ylim(-0.3,0.3);
    total_timedelta = tmax-tmin
    third_of_timedelta = total_timedelta/3
    plt.xticks([tmin,tmin+third_of_timedelta, tmin+ 2*third_of_timedelta, tmax])
    plt.xlabel('Date',fontsize=16)
    plt.ylabel('Vertical Current Shear (1/s)',fontsize=16)
    plt.gca().invert_yaxis()

    ##APPLY THE CTD DATA
    ax = plt.scatter(time,ADCP_var,s=1,c=depth,cmap='jet')

    #add a colorbar
    fig.colorbar(ax).set_label(label = 'Depth (m)',size=15,weight='bold')

    ##SAVE THE FIGURE:
    if not os.path.exists(fig_dir+'/2d/'):
        os.makedirs(fig_dir+'/2d/')

    print('Saving Output Image:  '+fig_dir+'2d/'+'ADCP_ShearVSTime_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png')
    plt.savefig(fig_dir+'2d/'+'ADCP_ShearVSTime_'+str(start_time.year)+str(start_time.month).zfill(2)+str(start_time.day).zfill(2)+str(start_time.hour).zfill(2)+str(start_time.minute).zfill(2)+'_'+str(end_time.year)+str(end_time.month).zfill(2)+str(end_time.day).zfill(2)+str(end_time.hour).zfill(2)+str(end_time.minute).zfill(2)+'.png',dpi='figure',format='png',bbox_inches='tight')
    #except: 
     #   print('There is no ADCP data from ' + str(start_date.year)+'/'+str(start_date.month).zfill(2)+'/'+str(start_date.day).zfill(2)+' '+str(start_date.hour).zfill(2)+':'+str(start_date.minute).zfill(2)+' to '+str(end_date.year)+'/'+str(end_date.month).zfill(2)+'/'+str(end_date.day).zfill(2)+' '+str(end_date.hour).zfill(2)+':'+str(end_date.minute).zfill(2))
       

In [ ]:
plot_ADCP_2D_depth(play1_start,play5_end)

## Make a Timeseries Line Plot of the ADCP Daa from a given time range at a Specific Depth

In [ ]:
def plot_ADCP_timeseries(play,depth):
    
    #SECLECT AND LOAD PLAY DATA
    play_start = play_start_dates[play-1]
    play_end = play_end_dates[play-1]
    
    time_data = np.squeeze(play_data_all[play-1]['time']).values
    depth_data = np.squeeze(play_data_all[play-1]['depth']).values
    shear_data = np.squeeze(play_data_all[play-1]['shear']).values
    
    depth_diff = abs(depth_data - depth)
    indx_depth = np.argmin(depth_diff)
    shear_data_at_depth = shear_data[indx_depth,:]

    #MAKE A TWO PANNELED TIME SERIES PLOT
    fig = plt.figure(figsize=(10,4))
    
    plt.plot(time_data,shear_data_at_depth,linewidth = 0.5,color='k')
    plt.ylabel('Vertical Current Shear (1/s)')
    plt.xlabel('Date')
    
    tmax = max(time_data)
    tmin = min(time_data)
    total_timedelta = tmax-tmin
    third_of_timedelta = total_timedelta/3
    plt.xticks([tmin,tmin+third_of_timedelta, tmin+ 2*third_of_timedelta, tmax])
    
    plt.ylim(-0.3,0.3)
    
    if play <=5:
        plt.title('Play '+str(play)+' Vertical Current Shear from Shipboard ADCP at '+str(depth)+'m')
    if play == 6: 
        plt.title('Vertical Current Shear from Shipboard ADCP (Entire Campaign) at '+str(depth)+'m')
        
    ##SAVE FIGURE 
    if not os.path.exists(fig_dir+'timeseries'):
        os.makedirs(fig_dir+'timeseries')
    if play==6: 
        print('Saving Output Image:  '+fig_dir+'timeseries/SASSIE_ADCP_VCS_Entire_Campaign_'+str(depth)+'m.png')
     #   plt.savefig(fig_dir+'timeseries/SASSIE_ADCP_VCS_Entire_Campaign.png',dpi='figure',format='png')
    else:
        print('Saving Output Image:  '+fig_dir+'timeseries/SASSIE_ADCP_VCS_play'+'_'+str(play)+'_'+str(depth)+'m.png')
      #  plt.savefig(fig_dir+'timeseries/SASSIE_ADCP_VCS_play'+'_'+str(play)+'.png',dpi='figure',format='png')
    

In [ ]:
plot_ADCP_timeseries(4,30)